- 드라이브 마운트 후 사용하기!
- 불러올 이미지, 저장할 이미지 경로변경하기
- 

In [ ]:
# 설치 후 런타임 재실행 
!pip install -U segmentation-models-pytorch albumentations --user
!pip install --user albumentations
!pip install opencv-contrib-python

In [ ]:
import albumentations 
from albumentations.pytorch import ToTensorV2
from typing import List
import torch
import numpy as np
import torch.nn as nn
import torchvision
from pathlib import Path
import albumentations as A
import torchvision.transforms as T
import segmentation_models_pytorch as smp
import os
import torchvision.transforms.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from matplotlib import colors, pyplot as plt
%matplotlib inline
from PIL import Image
import warnings
warnings.filterwarnings("ignore")
from skimage.filters import threshold_otsu
from torchvision.utils import draw_segmentation_masks
from tqdm import tqdm
import torchvision.transforms as transforms

In [ ]:
class test_NailsDataset(torch.utils.data.Dataset):
    def __init__(self, images: List[str], transform):
        self._images = images
        # self._masks = masks
        self._transform = transform
        # assert len(images) == len(masks)
  
    def __len__(self):
        return len(self._images)
  
    def __getitem__(self, index):
        image = Image.open(self._images[index])
        augmented = self._transform(image=np.array(image))
        image= augmented['image']
        return image

In [ ]:
def iou(target, prediction):
    prediction = prediction
    intersection = np.logical_and(target, prediction)
    union = np.logical_or(target, prediction)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

def iou2(target, prediction):
    tp = torch.sum(preds * y)  # TP
    fp = torch.sum(preds * (1 - y))  # FP
    fn = torch.sum((1 - preds) * y)  # FN
    tn = torch.sum((1 - preds) * (1 - y))  # TN

    iou_score2 = (tp + 1e-8) / (tp + fp + fn + 1e-8)
    mean_acc = (tp + 1e-8)/(tp+fn + 1e-8)
    
    return iou_score2, mean_acc

### 함수 정의

In [ ]:
def test_dataset(test_image):
  test_dataset = test_NailsDataset(
      images=test_image,
      transform=A.Compose([
          A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
          A.Resize(height= 224 ,width= 224), # 사이즈 변경하면 오류
          ToTensorV2(transpose_mask=True),
      ]),
      )
  
  test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1 ,shuffle=False)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  iou_score,iou_score2,mean_acc,otsu = 0, 0, 0, 0
  for i, imgs in tqdm(enumerate(test_dataloader)):
      for img in imgs:
          img = torch.unsqueeze(img, 0)  # torch.unsqueeze(): 1차원 제거 함수
          pred = best_model((img).to(device))[0][0].cpu().detach().numpy() > 0.47  
          preds = (best_model((img).to(device))[0][0].cpu().detach() > 0.47).float()

          # 시각화 
          fig, axes = plt.subplots(1, 2)
          axes[0].imshow(pred)
          axes[0].set_title("Predicted mask")

          img_o = ((img).to(device))[0][0].cpu().detach().numpy()
          axes[1].imshow(img_o)
          axes[1].set_title("Target image")

          fig.set_figwidth(8)   
          fig.set_figheight(4)  
          plt.show()

          # iou_score += iou(((pred).to(device))[0][0].cpu().detach().numpy(),pred)
          # print("IoU", iou(((pred).to(device))[0][0].cpu().detach().numpy(),pred))
          
          # 마스크 이미지 저장 
          # plt.imsave( f'model_mask_{i}.jpg', pred, cmap='gray') # 경로 변경

# 학습에 사용된 이미지로 결과 확인

### 손

In [ ]:
# 학습에 사용된 이미지
img_path = '/content/drive/MyDrive/4조/2.데이터/손이미지/'
test_hand_lst = sorted([img for img in os.listdir(img_path)])
test_hand_images = sorted([img_path + img for img in test_hand_lst])
len(test_hand_images), test_hand_lst[:3], test_hand_images[:3]

In [ ]:
# 모델 1
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_foot_seg_best_model.pth') # 첫번째 모델 사용

# 결과 확인
for g, p in zip(test_hand_lst, test_hand_images): 
  print(f' - img name: {g} ')
  test_hand_images_r  = [p]
  test_dataset(test_hand_images_r)
print('-----------'*30)

### 발

In [ ]:
# 학습에 사용된 이미지
img_path = '/content/drive/MyDrive/4조/2.데이터/발이미지/'
test_foot_lst = sorted([img for img in os.listdir(img_path)])
test_foot_images = sorted([img_path + img for img in test_foot_lst])
len(test_foot_images), test_foot_lst[:3], test_foot_images[:3]

In [ ]:
# 모델 1
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_foot_seg_best_model.pth') # 첫번째 모델 사용

# 결과 확인
for g, p in zip(test_foot_lst, test_foot_images): 
  print(f' - img name: {g} ')
  test_foot_images_r  = [p]
  test_dataset(test_foot_images_r)
print('-----------'*30)

# 실제 이미지로 결과 확인 

## 손

In [ ]:
img_path = '/content/drive/MyDrive/프포젝트/3_DDD/data/테스트이미지/손/'
test_hand_images =[img_path + img for img in os.listdir(img_path)]

In [ ]:
test_hand_images

In [ ]:
# 돌아서 불려오는 이미지 회전 후 저장
import cv2

new_path = '/content/' # 저장 경로 변경하기
new_file=[]
for i, f in enumerate(test_hand_images):
  file = plt.imread(f)
  if file.shape[0] < file.shape[1]:
    file = cv2.rotate(file, cv2.ROTATE_90_CLOCKWISE) # 돌아서 불려오는 이미지 회전 
    plt.imsave(new_path + f'hand_{i}.jpg', file )    # 저장 경로 변경하기
    new_file.append(f'hand_{i}.jpg')
    plt.imshow(file)
    plt.show()
new_file_path = [new_path + f for f in new_file] # 경로로 저장해야 파이토치 모델에서 불러올 수 있음 

In [ ]:
# 돌아서 불려오는 이미지 회전 후 저장
import cv2

new_path = '/content/' # 저장 경로 변경하기
new_file=[]
for i, f in enumerate(test_hand_images):
  file = plt.imread(f)
  if file.shape[0] < file.shape[1]:
    file = cv2.rotate(file, cv2.ROTATE_90_CLOCKWISE) # 돌아서 불려오는 이미지 회전 
    plt.imsave(new_path + f'hand_{i}.jpg', file )    # 저장 경로 변경하기
    new_file.append(f'hand_{i}.jpg')
    plt.imshow(file)
    plt.show()
new_file_path = [new_path + f for f in new_file] # 경로로 저장해야 파이토치 모델에서 불러올 수 있음 

In [ ]:
best_model = torch.load('/content/drive/MyDrive/프포젝트/3_DDD/data/nail_foot_seg_best_model.pth') # 오잉

for i in range(3):
  for p in new_file_path: 
    print(f' - {i+1} 번째 ')
    new_file_r  = [p]
    test_dataset(new_file_r)

  print('-----------'*30) /content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_foot_seg_best_model_2.pth

In [ ]:
# 모델 2
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_foot_seg_best_model_2.pth') # 오잉

for i in range(3):
  for p in new_file_path: 
    print(f' - {i+1} 번째 ')
    new_file_r  = [p]
    test_dataset(new_file_r)

  print('-----------'*30) 

In [ ]:
# 모델 3
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_seg_best_model_3.pth') # 오잉

for i in range(3):
  for p in new_file_path: 
    print(f' - {i+1} 번째 ')
    new_file_r  = [p]
    test_dataset(new_file_r)

  print('-----------'*30) 

In [ ]:
# 모델 4
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_seg_best_model_4.pth') # 오잉

for i in range(3):
  for p in new_file_path: 
    print(f' - {i+1} 번째 ')
    new_file_r  = [p]
    test_dataset(new_file_r)

  print('-----------'*30)

In [ ]:
plt.imshow(new_file[0])

In [ ]:
import cv2
pp = plt.imread(test_hand_images[0])
plt.imshow(pp)
pp.shape

In [ ]:
pp = cv2.imread(test_hand_images[0])
if pp.shape[0]<pp.shape[1]:
  pp = cv2.rotate(pp, cv2.ROTATE_90_CLOCKWISE)
plt.imshow(pp)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(pp, batch_size=1 ,shuffle=False)
test_dataloader

In [ ]:
best_model = torch.load('/content/drive/MyDrive/프포젝트/3_DDD/data/nail_foot_seg_best_model.pth') # 오잉

for i in range(3):
  for p in test_hand_images: 
    print(f' - {i+1} 번째 ')
    test_hand_images_r  = [p]
    test_dataset(test_hand_images_r)

  print('-----------'*30)

In [ ]:
# 학습 데이터가 적은 모델 
best_model = torch.load('/content/drive/MyDrive/프포젝트/3_DDD/best_model.pth') # 오잉 2

for i in range(3):
  for p in test_hand_images: 
    print(f' - {i+1} 번째 ')
    test_hand_images_r  = [p]
    test_dataset(test_hand_images_r)
  print('-----------'*30)

### 손 모델 1

In [ ]:
# 모델 1
# 저장된 모델 불러오기
# pytorch model 확장자 .pth
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_foot_seg_best_model.pth') # 첫번째 모델 사용

In [ ]:
len(test_hand_images)

In [ ]:
# 모델 1
for i in range(3):
  for p in test_hand_images: 
    print(f' - {i+1} 번째 ')
    test_hand_images_r  = [p]
    test_dataset(test_hand_images_r)
  print('-----------'*30)

In [ ]:
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_foot_seg_best_model.pth') # 1번째 모델

for i in range(3):
  for p in test_hand_images: 
    print(f' - {i+1} 번째 ')
    test_hand_images_r  = [p]
    test_dataset(test_hand_images_r)
  print('-----------'*30)

### 손 모델 2

In [ ]:
# 모델 2
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_foot_seg_best_model_2.pth') # 첫번째 모델 사용

In [ ]:
# 모델 2
for i in range(3):
  for p in test_hand_images: 
    print(f' - {i+1} 번째 ')
    test_hand_images_r  = [p]
    test_dataset(test_hand_images_r)
  print('-----------'*30)

### 손 모델 3 (손만 학습한 모델)

In [ ]:
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_seg_best_model_3.pth')

In [ ]:
# 모델 3
for i in range(3):
  for p in test_hand_images: 
    print(f' - {i+1} 번째 ')
    test_hand_images_r  = [p]
    test_dataset(test_hand_images_r)
  print('-----------'*30)

### 손 모델 4 (손만 학습한 모델)

In [ ]:
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_seg_best_model_4.pth')

In [ ]:
# 모델 4
for i in range(3):
  for p in test_hand_images: 
    print(f' - {i+1} 번째 ')
    test_hand_images_r  = [p]
    test_dataset(test_hand_images_r)
  print('-----------'*30)

## 발 

In [ ]:
img_path = '/content/drive/MyDrive/프포젝트/3_DDD/data/테스트이미지/발/'
test_foot_images =[img_path + img for img in os.listdir(img_path)]
test_foot_images

### 발 모델 1 (손,발 같이 학습한 모델)

In [ ]:
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_foot_seg_best_model.pth') # 1번째 모델

for i in range(3):
  for p in test_foot_images: 
    print(f' - {i+1} 번째 ')
    test_foot_images_r  = [p]
    test_dataset(test_foot_images_r)
  print('-----------'*30)

### 발 모델 2 (손,발 같이 학습한 모델)

In [ ]:
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_foot_seg_best_model_2.pth') # 1번째 모델

for i in range(3):
  for p in test_foot_images: 
    print(f' - {i+1} 번째 ')
    test_foot_images_r  = [p]
    test_dataset(test_foot_images_r)
  print('-----------'*30)

### 발 모델 3 (발만 학습한 모델)

In [ ]:
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/foot_seg_best_model_1.pth') # 1번째 모델

for i in range(3):
  for p in test_foot_images: 
    print(f' - {i+1} 번째 ')
    test_foot_images_r  = [p]
    test_dataset(test_foot_images_r)
  print('-----------'*30)